In [ ]:
from google.colab import drive
drive.mount('/content/data')

Mounted at /content/data


In [ ]:
import os
import numpy as np
import cv2
import random
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Function to preprocess image
def preprocess_image(image_path):
    # Loading and preprocessing image using OpenCV
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resizing the image to match ResNet input size
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB
    image = image.astype(np.float32) / 255.0  # Normalizing pixel values
    return image

# Function to parse the label file
def parse_label_file(label_file_path):
    labels = []
    with open(label_file_path, 'r') as label_file:
        for line in label_file:
            values = line.split()
            class_index = int(values[0])
            labels.append(class_index)
    return labels

# Function to load dataset
def load_dataset(data_dir, num_classes, test=False):
    images = []
    labels = []

    # Paths to images and their corresponding label files
    images_dir = os.path.join(data_dir, "images")
    labels_dir = os.path.join(data_dir, "labels")

    for image_name in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_name)
        # Preprocessing the original image
        original_image = preprocess_image(image_path)
        images.append(original_image)

        # Reading labels from the corresponding label file if it exists
        label_file_path = os.path.join(labels_dir, os.path.splitext(image_name)[0] + ".txt")
        if os.path.exists(label_file_path):
            label_vector = parse_label_file(label_file_path)
            labels.append(label_vector)
        else:
            # If the label file is missing, mark the label as an empty list
            labels.append([])

    # Converting the labels to binary format using MultiLabelBinarizer
    mlb = MultiLabelBinarizer(classes=range(num_classes))
    labels = mlb.fit_transform(labels)

    return np.array(images), labels

# Specifying the path dataset directories and the number of classes
train_data_dir = "/content/data/My Drive/AI_Project/train"
test_data_dir = "/content/data/My Drive/AI_Project/test"
val_data_dir = "/content/data/My Drive/AI_Project/valid"
num_classes = 10

# Loading and preprocessing the training dataset
train_images, train_labels = load_dataset(train_data_dir, num_classes)
print("Shape of training images array:", train_images.shape)
print("Shape of training labels array:", train_labels.shape)

# Loading and preprocessing the validation dataset
val_images, val_labels = load_dataset(val_data_dir, num_classes)
print("Shape of validation images array:", val_images.shape)
print("Shape of validation labels array:", val_labels.shape)

# Loading and preprocessing the test dataset
test_images, test_labels = load_dataset(test_data_dir, num_classes, test=True)
print("Shape of test images array:", test_images.shape)
print("Shape of test labels array:", test_labels.shape)


Shape of training images array: (1012, 224, 224, 3)
Shape of training labels array: (1012, 10)
Shape of validation images array: (230, 224, 224, 3)
Shape of validation labels array: (230, 10)
Shape of test images array: (107, 224, 224, 3)
Shape of test labels array: (107, 10)


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
32/32 [==============================] - 838s 25s/step - loss: 959.4907 - accuracy: 0.2984 - val_loss: 3346.5056 - val_accuracy: 0.4217
Epoch 2/10
32/32 [==============================] - 782s 24s/step - loss: 3158.7524 - accuracy: 0.2520 - val_loss: 1418.5485 - val_accuracy: 0.1957
Epoch 3/10
32/32 [==============================] - 775s 24s/step - loss: 5340.3491 - accuracy: 0.2559 - val_loss: 38579.2773 - val_accuracy: 0.1957
Epoch 4/10
32/32 [==============================] - 768s 24s/step - loss: 12936.8223 - accuracy: 0.2579 - val_loss: 18302.9551 - val_accuracy: 0.1957
Epoch 5/10
32/32 [==============================] - 767s 24s/step - loss: 25561.6562 - accuracy: 0.2727 - val_loss: 67.7590 - val_accuracy: 0.3652
Epoch 6/10
32/32 [==============================] - 769s 24s/step - loss: 35767.5625 - accuracy: 0.3686 - val_loss: 49.5805 - val_accuracy: 0.4348
Epoch 7/10
32/32 [=============================

In [ ]:
# learning rate 0.001
# batchsize = 32
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Loading the pre-trained ResNet model without classifier head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Adding a custom classifier head
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(256, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(128, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(64, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
output = Dense(num_classes, activation='softmax')(x)


# Creating final model
model = Model(inputs=base_model.input, outputs=output)

# Compiling model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Defining early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model with early stopping
history = model.fit(train_images, train_labels, epochs=30, batch_size=32,
                    validation_data=(val_images, val_labels), callbacks=[early_stopping])

# Evaluating the model on validation data
val_loss, val_accuracy = model.evaluate(val_images, val_labels)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Testing the model on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/30
32/32 [==============================] - 255s 8s/step - loss: 25.6556 - accuracy: 0.1670 - val_loss: 3.6140 - val_accuracy: 0.3696
Epoch 2/30
32/32 [==============================] - 248s 8s/step - loss: 9.7673 - accuracy: 0.2115 - val_loss: 4.7152 - val_accuracy: 0.3696
Epoch 3/30
32/32 [==============================] - 295s 9s/step - loss: 78.5749 - accuracy: 0.2520 - val_loss: 149.1138 - val_accuracy: 0.3696
Epoch 4/30
32/32 [==============================] - 248s 8s/step - loss: 3359.0999 - accuracy: 0.2421 - val_loss: 6275.8545 - val_accuracy: 0.3696
Epoch 5/30
32/32 [==============================] - 296s 9s/step - loss: 66594.5938 - accuracy: 0.2381 - val_loss: 91362.0469 - val_accuracy: 0.3696
Epoch 6/30
8/8 [==============================] - 41s 5s/step - loss: 3.6140 - accuracy: 0.3696
Validation Loss: 3.6140222549438477
Validation Accuracy: 0.3695652186870575
4/4 [==============================] - 1

In [ ]:
# learning rate 0.01
# batchsize = 32
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Loading the pre-trained ResNet model without classifier head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Adding a custom classifier head
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(256, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(128, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(64, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
output = Dense(num_classes, activation='softmax')(x)


# Creating final model
model = Model(inputs=base_model.input, outputs=output)

# Compiling model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Defining early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model with early stopping
history = model.fit(train_images, train_labels, epochs=30, batch_size=32,
                    validation_data=(val_images, val_labels), callbacks=[early_stopping])

# Evaluating the model on validation data
val_loss, val_accuracy = model.evaluate(val_images, val_labels)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Testing the model on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


NameError: name 'num_classes' is not defined

In [5]:
# learning rate 0.001
# Batch size 16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Loading the pre-trained ResNet model without classifier head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Adding a custom classifier head
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(256, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(128, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(64, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
output = Dense(num_classes, activation='softmax')(x)


# Creating final model
model = Model(inputs=base_model.input, outputs=output)

# Compiling model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Defining early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model with early stopping
history = model.fit(train_images, train_labels, epochs=30, batch_size=16,
                    validation_data=(val_images, val_labels), callbacks=[early_stopping])

# Evaluating the model on validation data
val_loss, val_accuracy = model.evaluate(val_images, val_labels)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Testing the model on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/30
64/64 [==============================] - 371s 6s/step - loss: 36.5539 - accuracy: 0.1976 - val_loss: 25.2588 - val_accuracy: 0.3696
Epoch 2/30
64/64 [==============================] - 342s 5s/step - loss: 2153.4602 - accuracy: 0.2441 - val_loss: 8572.5166 - val_accuracy: 0.3696
Epoch 3/30
64/64 [==============================] - 340s 5s/step - loss: 430896.1562 - accuracy: 0.2540 - val_loss: 957462.6875 - val_accuracy: 0.4087
Epoch 4/30
64/64 [==============================] - 340s 5s/step - loss: 9313203.0000 - accuracy: 0.2372 - val_loss: 14357632.0000 - val_accuracy: 0.3696
Epoch 5/30
64/64 [==============================] - 346s 5s/step - loss: 54879372.0000 - accuracy: 0.2164 - val_loss: 60991960.0000 - val_accuracy: 0.3696
Epoch 6/30
8/8 [==============================] - 48s 6s/step - loss: 25.2588 - accuracy: 0.3696
Validation Loss: 25.258764266967773
Validation Accuracy: 0.3695652186870575
4/4 [==============================] - 23s 5s/step - loss: 73.5548 - accuracy

In [ ]:
# learning rate 0.01
# Batch size 16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Loading the pre-trained ResNet model without classifier head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Adding a custom classifier head
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(256, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(128, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(64, activation='relu')(x)  # Adding another dense layer
x = Dropout(0.5)(x)  # Adding dropout for regularization
output = Dense(num_classes, activation='softmax')(x)


# Creating final model
model = Model(inputs=base_model.input, outputs=output)

# Compiling model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Defining early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model with early stopping
history = model.fit(train_images, train_labels, epochs=30, batch_size=16,
                    validation_data=(val_images, val_labels), callbacks=[early_stopping])

# Evaluating the model on validation data
val_loss, val_accuracy = model.evaluate(val_images, val_labels)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Testing the model on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
